In [ ]:
scp Pool9_Barcodes.txt sarahgw@amphiprion.deenr.rutgers.edu:/local/home/sarahgw/WGS/Sequences/Pool9/

In [ ]:
mkdir $(awk '{print $1}' /local/home/sarahgw/WGS/Sequences/Pool9/Pool9_Barcodes.txt)

In [1]:
!ls /local/home/sarahgw/WGS/Sequences/Pool9/

KY_E_14  KY_E_17  KY_E_20  KY_E_23  KY_E_26  KY_U_12  KY_U_16  KY_U_22
KY_E_15  KY_E_18  KY_E_21  KY_E_24  KY_U_10  KY_U_13  KY_U_17  NY_U_08
KY_E_16  KY_E_19  KY_E_22  KY_E_25  KY_U_11  KY_U_15  KY_U_18  NY_U_22


In [ ]:
gunzip */*.fastq.gz

In [1]:
!grep -c '@' /local/home/sarahgw/WGS/Sequences/Pool9/*/*.fastq

/local/home/sarahgw/WGS/Sequences/Pool9/KY_E_14/KY_E_14_Demultiplexed-fastq-fileset-sample-Pool-9-from-flowcell-HW3FCBCXY-on-2017-10-09_Read_1_passed_filter.fastq:180395
/local/home/sarahgw/WGS/Sequences/Pool9/KY_E_14/KY_E_14_Demultiplexed-fastq-fileset-sample-Pool-9-from-flowcell-HW3FCBCXY-on-2017-10-09_Read_4_passed_filter.fastq:222867
/local/home/sarahgw/WGS/Sequences/Pool9/KY_E_14/KY_E_14_FU1.fastq:179718
/local/home/sarahgw/WGS/Sequences/Pool9/KY_E_14/KY_E_14_FU2.fastq:221956
/local/home/sarahgw/WGS/Sequences/Pool9/KY_E_15/KY_E_15_Demultiplexed-fastq-fileset-sample-Pool-9-from-flowcell-HW3FCBCXY-on-2017-10-09_Read_1_passed_filter.fastq:2117524
/local/home/sarahgw/WGS/Sequences/Pool9/KY_E_15/KY_E_15_Demultiplexed-fastq-fileset-sample-Pool-9-from-flowcell-HW3FCBCXY-on-2017-10-09_Read_4_passed_filter.fastq:2645171
/local/home/sarahgw/WGS/Sequences/Pool9/KY_E_15/KY_E_15_FU1.fastq:2052335
/local/home/sarahgw/WGS/Sequences/Pool9/KY_E_15/KY_E_15_FU2.fastq:2558802
/local/home/sarahgw/WGS/

In [ ]:
~/Notebooks/WNS_WGS/Bioinfo_pipeline.sh 

In [ ]:
rm */*passed_filter.fastq
    rm */*FU1.fastq
    rm */*FU2.fastq
    rm */*trim_p.fastq
    rm */*trim_up.fastq
find . -name "*.tagged.fastq" -exec mv -i {} ../Pool8Contaminants/ \;
    for file in $(ls); do tar -zcvf "$file".tar.gz "$file" ; done

In [ ]:
for dir in $(ls); do cd $dir; /local/home/sarahgw/anaconda2/bin/bowtie2 --very-sensitive-local -p 12 -x /local/home/sarahgw/references/Myotis_Lucifugus2.0/MyoLuc_toplevel_index -1 $(pwd)/"$dir"_FU1_trim_p.tagged_pfilter.fastq -2 $(pwd)/"$dir"_FU2_trim_p.tagged_pfilter.fastq -U $(pwd)/"$dir"_FU1_trim_up.tagged_filter.fastq,$(pwd)/"$dir"_FU2_trim_up.tagged_filter.fastq -S $(pwd)/"$dir"_pairedplus_eg2.sam; cd ..; done

In [ ]:
for dir in $(ls); do cd $dir; java -jar /local/home/sarahgw/picard.jar SortSam \
      I=$(pwd)/"$dir"_pairedplus_eg2.sam \
      O=$(pwd)/"$dir"_pairedplus_eg2_sorted.bam \
      SORT_ORDER=coordinate; cd ..; done

In [ ]:
for dir in $(ls); do cd $dir; java -jar /local/home/sarahgw/picard.jar MarkDuplicates \
      I=$(pwd)/"$dir"_pairedplus_eg2_sorted.bam \
      O=$(pwd)/"$dir"_pairedplus_marked_dupes.bam \
      M=$(pwd)/"$dir"_pairedplus_marked_dup_metrics.txt \
      REMOVE_DUPLICATES=true \
      ASSUME_SORT_ORDER=coordinate; cd ..; date; done 

In [ ]:
for dir in $(ls); do cd $dir; samtools view -b $(pwd)/"$dir"_pairedplus_marked_dupes.bam | genomeCoverageBed -ibam stdin -g /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myoluc_genomeFile.txt>$(pwd)/"$dir"_Coverage.txt; cd ..; done